In [1]:
import numpy as np
from fastapi import FastAPI, responses
from pydantic import BaseModel
from configparser import ConfigParser
from util import get_model,get_encoder
import pickle

from embedders.classification.contextual import TransformerSentenceEmbedder


In [2]:
use_encoder = 'True'
transformer = TransformerSentenceEmbedder('distilbert-base-uncased')


# Load in the data, lower all words
corpus = ['test', 'test2', 'test3']

# Create embeddings to make text usable by ml model
embeddings = transformer.transform(corpus)

# Use ml model to create predictions
model = pickle.load(open("/home/leonardpuettmann/repos/automl-docker/ml/model.pkl", "rb"))

predictions = model.predict(embeddings).tolist()

probabilities = model.predict_proba(embeddings)
probabilities_max = [round(np.max(i), 2) for i in probabilities]
print(predictions)
print(probabilities_max)



Some weights of the model checkpoint at /home/leonardpuettmann/.cache/torch/sentence_transformers/distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initializing model, might take some time...


Encoding batches ...: 100%|██████████| 1/1 [00:00<00:00, 41.98it/s]

[1, 2, 1]
[0.56, 0.42, 0.43]


In [4]:
results = []
if use_encoder == 'True':
    encode = pickle.load(open("/home/leonardpuettmann/repos/automl-docker/ml/encoder.pkl", "rb"))
    predictions_labels = encode.inverse_transform(predictions).tolist()

    for i, j in zip(predictions_labels, probabilities_max):
        results.append({'label': i, 'confidence': j})   
else:
    for i, j in zip(predictions, probabilities_max):
        results.append({'label': i, 'confidence': j})
print(results)

[{'label': 'neutral', 'confidence': 0.56}, {'label': 'positive', 'confidence': 0.42}, {'label': 'neutral', 'confidence': 0.43}]
